In [1]:
import csv
#import os
import numpy
import math
import xlwt
# import xlsxwriter
import pandas as pd
from numpy.linalg import inv
import time
import matplotlib.pyplot as plt

### IMPORT DATA AS LIST

## FIXED VARIABLES

In [22]:
#--------------- Fixed Variables------------------#
annealingRate = 0.96 # Annealing rate
alpha = 0.0001
multiplier_lambda2 = 200 #lambda is beta in the paper
# T_final = 100 # Final temperature to stop the annealing process
# T_initial = 2.5
    
nX = len(move_data) # Number of features from ILI 1
nY = len(fix_data) # Number of features from ILI 2
d = 3 # Coordinate dimensions



22


## Function 1: Distance of All feature Pairs

In [9]:
def Distance_Matrix (nX, nY, x, y):
    distance = numpy.zeros((nX, nY))
    for i in range(nX):
        for j in range(nY):
            distance[i][j] = math.sqrt(sum((numpy.subtract(x[i],y[j]))**2 ))
    return distance

## Function 2: P matrix calculation

In [10]:
# P matrix calculation
def DetermineCorrespondenceMatrix(nX, nY,  x, y, T, T_outlier, alpha):
    # x = numpy.zeros((nX, d))
    # y = numpy.zeros((nY, d))
    p = numpy.zeros((nX+1, nY+1))
    # distFeatPairs = numpy.zeros((nX, nY))
    # outlierCenter = numpy.zeros(d)

    # Normal assignment
    distFeatPairs = Distance_Matrix(nX, nY,  x, y) # Distance of all feature pairs
    p = numpy.exp((alpha - (distFeatPairs**2) )/ T - math.log( T ))

    #  Outliers in x
    outlierCenter_y = numpy.asarray(y).sum(axis=0) / nY
    x_outlier_dist = []
    for i in range(nX):
        # j = nY
        x_outlier_dist.append(sum( (x[i] - outlierCenter_y)**2 ))
    x_outlier=numpy.exp(- numpy.array(x_outlier_dist) / T_outlier - math.log( T_outlier ))

    p_with_xoutlier = numpy.c_[p,x_outlier]

    # # Outliers in y
    outlierCenter_x = numpy.asarray(x).sum(axis=0) / nX 
    y_outlier_dist = []
    for j in range(nY):
        # i = nX
        y_outlier_dist.append(sum( (outlierCenter_x - y[j] )**2 ))
    y_outlier = numpy.exp(- numpy.array(y_outlier_dist) / T_outlier - math.log( T_outlier ))
    
    y_outlier = numpy.append(y_outlier,0)

    p_matrix = numpy.vstack((p_with_xoutlier,y_outlier))
    return p_matrix


## Function 3: Normalize P Matrix

In [11]:
def Normalize_P(nX, nY, p_matrix):
    threshold = 1e-04
    # rowsum = numpy.zeros((nX))
    # rowsum=numpy.array(rowsum)
    # colsum = numpy.zeros((nY))
    # colsum=numpy.array(colsum)
    converged = False

    while converged == False:       
        # sum of columns and rows
        colsum = numpy.sum(p_matrix[:,0:nY],axis=0)
        # Initial normalize on columns
        for j in range(nY):
            if (colsum[j] > 0.0):
                p_matrix[:,j] = p_matrix[:,j] / colsum[j]  ####Check shavad dimension[63,37]
        # Initial Normalize on rows
        rowsum = numpy.sum(p_matrix[0:nX,:],axis=1)
        
        for i in range(nX):
            if (rowsum[i] > 0.0):
                p_matrix[i,:]=p_matrix[i,:]/rowsum[i]

        colsum2 = numpy.sum(p_matrix[:,0:nY],axis = 0 )
        if (abs(colsum2-1)>threshold).any():
            converged = False
        else:
            converged = True
            p_matrix=p_matrix.round(2)
            return p_matrix


# Function 4: Determine Optimal Transformation

In [12]:
def DetermineOptimalTransformation(nX, nY, x, y, p, lambda2):
    x_array = numpy.asarray(x)
    y_array = numpy.asarray(y)
    p_red = p[0:nX, 0:nY] # Reduced p-matrix
    y1 = numpy.tile(y_array[:,0],(nX,1)) # 62*37:#(move_data)*#(fix_data), each column is the distance of fix_data
    y2 = numpy.tile(y_array[:,1],(nX,1)) # 62*37:#(move_data)*#(fix_data), each column is the orientation of fix_data
    #print(y1.shape)
    x1 = numpy.transpose(numpy.tile(x_array[:,0],(nY,1))) # 62*37:#(move_data)*#(fix_data), each row is the distance of move_data
    x2 = numpy.transpose(numpy.tile(x_array[:,1],(nY,1))) # 62*37:#(move_data)*#(fix_data), each row is the orientation of move_data

    ## Affine
    a_11 = numpy.sum(p_red*x1*x1)+lambda2 # element multiplication and summation
    a_12 = numpy.sum(p_red*x1*x2)
    a_13 = numpy.sum(p_red*x1)
    a_21 = a_12
    a_22 = numpy.sum(p_red*x2*x2)+lambda2
    a_23 = numpy.sum(p_red*x2)
    a_31 = a_13
    a_32 = a_23
    a_33 = numpy.sum(p)+lambda2

    A = numpy.asarray([[a_11,a_12,a_13],[a_21,a_22,a_23],[a_31,a_32,a_33]]) #3*3, symmatric 
    
    B1 = numpy.asarray([numpy.sum(p_red*y1*x1)+lambda2,numpy.sum(p_red*y1*x2),numpy.sum(p_red*y1)]) #(3,)
    
    B2 = numpy.asarray([numpy.sum(p_red*y2*x1),numpy.sum(p_red*y2*x2)+lambda2,numpy.sum(p_red*y2)]) #(3,)
    
    affine_1 = numpy.dot(inv(A),B1) #matrix multiplication
    affine_2 = numpy.dot(inv(A),B2)
    affine_3 = numpy.asarray([0,0,1])

    affine = numpy.asarray([affine_1,affine_2,affine_3])
    #print(affine.shape)
    #print(affine)

    ## Determine the Energy function
    x = numpy.transpose(numpy.c_[x,numpy.ones(nX)])
    x_trans = numpy.transpose(numpy.matmul(affine,x))
    #print(x_trans.shape)
    x_trans = x_trans[:,0:1]
    #print(x_trans.shape)
    distFeatPairs = Distance_Matrix(nX,nY,x_trans,y)
    E = numpy.sum(p_red*distFeatPairs*distFeatPairs)

    return affine,E
  

## Main process

In [23]:
## Set Temperatures for analysis
start = time.time()
distFeatPairs=Distance_Matrix (nX, nX, move_data,move_data)
T_init = (numpy.max(distFeatPairs))**2
distlarger0= distFeatPairs[distFeatPairs>0]
T_final = (numpy.min(distlarger0))**2
T_outlier = T_init
T = T_init
E_temp = []
## Set initial affine
affine = numpy.array(numpy.eye(d, dtype = float))

counter = 0

while T > T_final:
    # print(T)
    lambda2 = multiplier_lambda2*T
    move_data_trans = numpy.transpose(numpy.c_[move_data,numpy.ones(nX)])
    x_trans = numpy.transpose(numpy.matmul(affine,move_data_trans))
    x_trans = x_trans[:,0:2]

    p_matrix = DetermineCorrespondenceMatrix(nX, nY,  move_data, fix_data, T, T_outlier, alpha)
    p_matrix = Normalize_P(nX,nY,p_matrix)
    
    if ( numpy.any(numpy.isnan(numpy.array(p_matrix))) == False ):
        affine,E= DetermineOptimalTransformation(nX, nY, move_data, fix_data, p_matrix, lambda2)
        E_temp.append([T,E])
        counter+=1
       #print(counter)
       #print(affine)
    else:
        break
    T_old = T
    p_old = p_matrix
    T = T* annealingRate

end = time.time()
#runtime in minutes
runtime = (end - start)/60
runtime = "%.2f"%runtime
print(runtime)
#numpy.savetxt(("p_matrix_%s.csv")%joint, p_matrix, delimiter=",")    
#numpy.savetxt(("affine_%s.csv")%joint, affine, delimiter=",")
numpy.savetxt(("x_trans_%s.csv")%joint, x_trans, delimiter=",")




(22, 20)
1
[[0.96882849 0.38909491 0.11671775]
 [0.03639615 0.34352664 0.01774002]
 [0.         0.         1.        ]]
(22, 20)
2
[[0.97209256 0.33279673 0.10992545]
 [0.03617076 0.34777473 0.01761163]
 [0.         0.         1.        ]]
(22, 20)
3
[[0.97492456 0.2924679  0.10438367]
 [0.03596676 0.35178524 0.01766658]
 [0.         0.         1.        ]]
(22, 20)
4
[[0.9775026  0.25533232 0.09703765]
 [0.03587885 0.35300389 0.01744768]
 [0.         0.         1.        ]]
(22, 20)
5
[[0.98287437 0.17249876 0.09017631]
 [0.03526524 0.36537256 0.01721261]
 [0.         0.         1.        ]]
(22, 20)
6
[[0.98382921 0.15718766 0.08787066]
 [0.03524999 0.36600326 0.01709796]
 [0.         0.         1.        ]]
(22, 20)
7
[[0.9911159  0.02802689 0.0811991 ]
 [0.03537238 0.36514292 0.01671491]
 [0.         0.         1.        ]]
(22, 20)
8
[[0.99302907 0.0074881  0.07818823]
 [0.03531502 0.36646947 0.01688173]
 [0.         0.         1.        ]]
(22, 20)
9
[[ 0.99974209 -0.10614591  0.

KeyboardInterrupt: 

Calculate match rate

In [37]:
# matching rate for move set 
num_outliers_x = sum(p_matrix[0:nX,nY]>0.9) # number of outliers
match_rate_move = 1-num_outliers_x/nX # matching reate over 90%
match_rate_move = "%.2f"%match_rate_move

# matching rate for fix set
num_outliers_y = sum(p_matrix[nX,0:nY]>0.9) # number of outliers
match_rate_fix = 1-num_outliers_y/nY # matching reate over 90%
match_rate_fix = "%.2f"%match_rate_fix

# ratio of sample sizes: fix/move
ratio_fix_to_move = nY/nX
ratio_fix_to_move = "%.2f"%ratio_fix_to_move

0.0


In [ ]:
# Function for building the Evaluation Table

from csv import writer
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)


In [ ]:
append_list_as_row('Evaluation Table.csv',[joint, nY, nX, ratio_fix_to_move, match_rate_fix, match_rate_move, runtime])

Plots

In [38]:
# Distance: DS_TO_US, Orientation: arch to 0 in meters
original_fix  = pd.DataFrame(fix_data, columns = ['Distance','Orientation'])
original_fix["category"] = "fix"
#move dataset: before transformation
original_move = pd.DataFrame(move_data,columns = ["Distance","Orientation"])
original_move["category"] = "move"


#move dataset: after transformation
adjust_move = pd.DataFrame(x_trans,columns = ["Distance","Orientation"])
adjust_move["category"] = "adjust_move"



# normalize the x and y axis
dist_min = original_fix["Distance"].min()
dist_max = original_fix["Distance"].max()
orien_min = original_fix["Orientation"].min()
orien_max = original_fix["Orientation"].max()
range_orientation = orien_max - orien_min
range_distance = dist_max - dist_min

original_fix["Orientation"] = (original_fix["Orientation"]- orien_min) / range_orientation
original_fix["Distance"] = (original_fix["Distance"] - dist_min) / range_distance
original_move["Orientation"] = (original_move["Orientation"]-orien_min) / range_orientation
original_move["Distance"] = (original_move["Distance"] - dist_min) / range_distance
adjust_move["Orientation"] = (adjust_move["Orientation"] - orien_min) / range_orientation
adjust_move["Distance"] = (adjust_move["Distance"] - dist_min) / range_distance

original_data = pd.concat([original_fix,original_move],axis = 0)
adjust_data = pd.concat([original_fix,adjust_move],axis = 0)


In [43]:
#import plotly at local anaconda first! 
# check website for instructions: https://stackoverflow.com/questions/36959782/plotly-in-jupyter-issue

import plotly.express as px
original_plot = px.scatter(original_data,x="Distance",y="Orientation",color ="category")
original_plot.write_image(file=('before_normalized_%s.png')%(joint))
original_plot.show()



ModuleNotFoundError: No module named 'plotly'

In [42]:
adjust_plot = px.scatter(adjust_data,x="Distance",y="Orientation",color ="category")
adjust_plot.write_image(file=('after_normalized_%s.png')%(joint))
adjust_plot.show()


NameError: name 'px' is not defined